In [1]:
import pandas as pd
from pandas import json_normalize
import json
import requests
import os
from unidiff import PatchSet
import git
from io import StringIO
import shutil
from git import Repo
import subprocess
import filecmp

In [2]:
with open("/home/ashwin/UIUC/CS527/Project/CS527-Project/Bugs/BugSwarm/Export.json", "r") as file:
    data = json.load(file)

In [3]:
# Assuming you already loaded the JSON data into a DataFrame
df = pd.read_json('/home/ashwin/UIUC/CS527/Project/CS527-Project/Bugs/BugSwarm/Export.json')
json_struct = json.loads(df.to_json(orient="records"))    

# Now, use json_normalize directly
df_flat = json_normalize(json_struct)

In [4]:
df_flat[df_flat['failed_job.job_id'] == 7068728014]

,image_tag,lang,diff_url,repo,ci_service,metrics.changes,reproducibility_status.status,failed_job.job_id,failed_job.build_id,failed_job.num_tests_run,failed_job.num_tests_failed,failed_job.trigger_sha,passed_job.trigger_sha,classification.build,classification.code,classification.exceptions,classification.test
2662,bobocode-projects-java-fundamentals-exercises-...,Java,https://github.com/bobocode-projects/java-fund...,bobocode-projects/java-fundamentals-exercises,github,2,Reproducible,7068728014,2567759045,180,1,9119545c430ed7119ad0b4e48cefacbeef41814c,af77c7c97f54fba8ccc68b088a6f4566ce0c3db0,No,Yes,[AssertionError],No


In [5]:
for record in data:
    if record.get('failed_job').get('job_id') == 7068728014:
        print(True)

True


In [6]:
def clone_repo_at_commit(repo_url, repo, failed_dest_dir, passed_dest_dir,  failed_commit_sha, passed_commit_sha, output_file_path):
    print(repo_url, repo.split('/')[1], failed_dest_dir, passed_dest_dir, output_file_path)
    subprocess.run(["git", "clone", repo_url, '/tmp/' + repo.split('/')[1] + '/Buggy'])
    os.chdir('/tmp/' + repo.split('/')[1] + '/Buggy')
    subprocess.run(["git", "reset", "--hard", failed_commit_sha])
    shutil.copytree('/tmp/' + repo.split('/')[1] + '/Buggy', failed_dest_dir, ignore=shutil.ignore_patterns('.git*'))

    subprocess.run(["git", "clone", repo_url, '/tmp/' + repo.split('/')[1] + '/Patched'])
    os.chdir('/tmp/' + repo.split('/')[1] + '/Patched')
    subprocess.run(["git", "reset", "--hard", passed_commit_sha])
    shutil.copytree('/tmp/' + repo.split('/')[1] + '/Patched', passed_dest_dir, ignore=shutil.ignore_patterns('.git*'))

    with open(output_file_path, 'w') as output_file:
            subprocess.run(['git', 'diff', failed_dest_dir, passed_dest_dir], stdout=output_file)

    subprocess.run(["rm", "-rf", "/tmp/" + repo.split('/')[1]])
    
    

In [7]:
input = ["apache-shiro-16456433682", 
    "bobocode-projects-java-fundamentals-exercises-7068728014",
    "alibaba-fastjson2-9745791070",
    "apache-servicecomb-java-chassis-9220678028",
    "apache-shardingsphere-15043584565",
    "apache-shardingsphere-14283109357",
    # "square-okhttp-397194600",
    #"TheAlgorithms-Python-17493072334",
    "kevin218-Eureka-12929091479",
    "spesmilo-electrum-293054995",
    "swagger-api-swagger-core-245823910",
    # "apache-jackrabbit-oak-337535542",
    # "spring-projects-spring-security-oauth-451380564",
    # "spring-projects-spring-security-oauth-451380565",
    "paramiko-paramiko-88695199",
    # "languagetool-org-languagetool-546883807",
    "traccar-traccar-9287925064",
    "joel-costigliola-assertj-core-309871149",
    "EducationalTestingService-skll-308660061",
    "EducationalTestingService-skll-308660063",
    #"debezium-debezium-15920299506",
    #"IHVH-OEMIB_PI01_19_TBOT-12579424042",
    #"LASY-org-lasy-13665357123",
    #"thm-projects-arsnova-backend-349234145",
    #"aws-aws-sam-cli-16196164089",
    #"graphhopper-graphhopper-221837923",
    # "SpigotMC-BungeeCord-10806539885",
    # "SpigotMC-BungeeCord-10806539922",
    # "SpigotMC-BungeeCord-10806539963",
    "alibaba-fastjson2-9745790591",
    "alibaba-fastjson2-9745790836",
    #"yt-dlp-yt-dlp-16047395836",
    #"IQSS-dataverse-402758963",
    "EducationalTestingService-skll-308660060",
    #"igniterealtime-Openfire-16538544714",
    #"NitorCreations-nflow-654160011",
    # "Adobe-Consulting-Services-acs-aem-commons-439308122",
    "saulpw-visidata-17624006231",
    "apache-dubbo-416671625"]

In [181]:
input = ["alibaba-fastjson2-13283643245"]

In [182]:
input = [i.split('-')[1:] for i in input]

In [183]:
job_name = ['-'.join(sublist[:]) for sublist in input]
job_id = [int(sublist[-1]) for sublist in input]

In [184]:
input = {job_name[i]: job_id[i] for i in range(len(job_name))}

In [185]:
for record in data:
    if record.get('failed_job').get('job_id') in input.values():
        failed_job = record.get("failed_job")
        root_dir = '/home/ashwin/UIUC/CS527/Project/CS527-Project/Bugs/BugSwarm'
        passed_job = record.get("passed_job")
        repo = record.get("repo")
        base_repo_url = 'https://github.com/'
        diff_url = record.get("diff_url")
        dest_dir = os.path.join(root_dir, list(input.keys())[list(input.values()).index(record.get('failed_job').get('job_id'))])
        os.mkdir(dest_dir)           
        os.chdir(dest_dir)
        
        repo_url = f"{base_repo_url}{repo}.git"
        if failed_job:
            failed_trigger_sha = failed_job.get("trigger_sha")
            failed_dest_dir = dest_dir + "/Buggy-version"
        
        if passed_job:
            passed_trigger_sha = passed_job.get("trigger_sha")
            passed_dest_dir = dest_dir + "/Patched-Version"
    
        output_file_path = dest_dir + f"/Diff"
        clone_repo_at_commit(repo_url, repo, failed_dest_dir, passed_dest_dir, failed_trigger_sha, passed_trigger_sha, output_file_path)
       

https://github.com/alibaba/fastjson2.git fastjson2 /home/ashwin/UIUC/CS527/Project/CS527-Project/Bugs/BugSwarm/fastjson2-13283643245/Buggy-version /home/ashwin/UIUC/CS527/Project/CS527-Project/Bugs/BugSwarm/fastjson2-13283643245/Patched-Version /home/ashwin/UIUC/CS527/Project/CS527-Project/Bugs/BugSwarm/fastjson2-13283643245/Diff


Cloning into '/tmp/fastjson2/Buggy'...


HEAD is now at 3924d7663 deserialize support OffsetDateTime


Cloning into '/tmp/fastjson2/Patched'...


HEAD is now at 9e9744985 improve OffsetDateTime support
